# Under the hood: Configure bia-bob's _behaviour_ through system messages

In this notebook we demonstrate how you can configure what kind of Python code bia-bob will generate. Note that the following configuration will be stored across sessions. When installing a new version of bia-bob, these settings are reset. You can also manually reset bia-bob's configuration by deleting the `.cache\bia-bob` directory in your home directory.

In [1]:
# load secret API key. You must unpack the contents of api_key.zip 
# into the same folder before going ahead.
from dotenv import load_dotenv
load_dotenv()

import os
from bia_bob import bob, DEFAULT_SYSTEM_PROMPT
bob.initialize(endpoint='https://llm.scads.ai/v1', model='meta-llama/Llama-3.3-70B-Instruct', api_key=os.environ.get('SCADSAI_API_KEY'))

## Custom system messages
You can configure your own system message. In the following example we enforce bob to always just answer a given question and never write code.

In [3]:
bob.initialize(
    endpoint='https://llm.scads.ai/v1', model='meta-llama/Llama-3.3-70B-Instruct', api_key=os.environ.get('SCADSAI_API_KEY'),
    system_prompt="""
    You are an excellent Python programmer who always 
    uses Finnish variable names in their code.
    """)

In [4]:
%bob Write a for-loop that prints the numbers from 0 to 10.

In [ ]:
# Loopin alku
for luku in range(11):
    # Tulosta luku
    print(luku)

## Default system message
You can reset the config to the default system message like this:

In [5]:
bob.initialize(
    endpoint='https://llm.scads.ai/v1', model='meta-llama/Llama-3.3-70B-Instruct', api_key=os.environ.get('SCADSAI_API_KEY'),
    system_prompt=DEFAULT_SYSTEM_PROMPT)

In [6]:
%bob Write a for-loop that prints the numbers from 0 to 10.

This task involves writing a simple for-loop in Python to print numbers from 0 to 10.



In [ ]:
import sys
for i in range(11):
    sys.stdout.write(str(i) + '\n')

## Domain specific system messages
You can instruct bia-bob to use common Python libraries from specific domains.

In [7]:
from bia_bob import bob
bob.initialize(
    endpoint='https://llm.scads.ai/v1', model='meta-llama/Llama-3.3-70B-Instruct', api_key=os.environ.get('SCADSAI_API_KEY'),
    system_prompt="""
    You are an excellent astronomer and Python programmer.
    You typically use Python libraris from this domain.
""")

In [8]:
%bob Write a function that can detect stars in an image.

**Star Detection Function**
=====================================

The following Python function utilizes the OpenCV library to detect stars in an astronomical image. This function applies a series of image processing techniques to enhance the visibility of stars and then uses a thresholding method to identify potential star locations.



In [ ]:
import cv2
import numpy as np
from astropy.io import fits
from photutils import DAOStarFinder

def detect_stars(image_path):
    """
    Detect stars in an astronomical image.

    Args:
        image_path (str): Path to the input image.

    Returns:
        A list of star coordinates (x, y) in the image.
    """
    # Load the image
    image = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Apply thresholding to enhance star visibility
    _, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Find contours of potential stars
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Initialize a list to store star coordinates
    star_coords = []

    # Iterate through contours and filter out non-star objects
    for contour in contours:
        area = cv2.contourArea(contour)
        x, y, w, h = cv2.boundingRect(contour)
        aspect_ratio = float(w)/h
        if area > 10 and aspect_ratio > 0.5 and aspect_ratio < 2:
            # Calculate the center of the contour
            M = cv2.moments(contour)
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            star_coords.append((cx, cy))

    return star_coords

# Example usage
image_path = 'astronomical_image.jpg'
star_coords = detect_stars(image_path)
print(star_coords)

def detect_stars_astropy(image_path):
    """
    Detect stars in an astronomical image using AstroPy.

    Args:
        image_path (str): Path to the input image (FITS file).

    Returns:
        A table of star coordinates and fluxes.
    """
    # Load the FITS image
    hdul = fits.open(image_path)
    image = hdul[0].data

    # Create a DAOStarFinder object
    daofind = DAOStarFinder(fwhm=3, threshold=5*image.std())

    # Find sources in the image
    sources = daofind(image)

    return sources

# Example usage
image_path = 'astronomical_image.fits'
sources = detect_stars_astropy(image_path)
print(sources)

## Terrible system messages
You can also instruct bia-bob to do stupid things. Be careful with this functionality.

In [9]:
bob.initialize(
    endpoint='https://llm.scads.ai/v1', model='meta-llama/Llama-3.3-70B-Instruct', api_key=os.environ.get('SCADSAI_API_KEY'),
    system_prompt="""
    You always write super complicated, hard-to-read Python code 
    surrounded with markdown fences. 
    with a lot of unnecessary loops and weird variable names. 
    Make sure it consists of at least 5 lines of code.
""")

In [10]:
%bob print hello world

In [ ]:
# Import the requisite modules
import sys

# Define a function to print the greeting
def print_greeting():
    # Print the greeting
    print("Hello world")

# Call the function, with a redundant check to ensure it's defined
if 'print_greeting' in globals():
    print_greeting()

In [11]:
# Reset default behaviour, as it would be stored permanently otherwise.
bob.initialize(
    endpoint='https://llm.scads.ai/v1', model='meta-llama/Llama-3.3-70B-Instruct', api_key=os.environ.get('SCADSAI_API_KEY'),
    system_prompt=DEFAULT_SYSTEM_PROMPT)

In [12]:
%bob print hello world

The task is to print "Hello World" to the console.



In [ ]:
import sys
print("Hello World")